$\textbf{IMPORTS}$

In [19]:
import requests
import numpy as np
import csv
import json
import time
import pprint
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

$\textbf{LOAD ALL PAGES}$, BOLIGA.DK

In [20]:
#1.Load pages boliga.dk
links = []
for page in range(1,148): #check the number of pages in link before running this code!!
    url_boliga = f'https://www.boliga.dk/resultat?propertyType=3&page={page}&sort=daysForSale-a'
    links.append(url_boliga)

$\textbf{SCRAPE DATA FROM EACH PAGE}$, BOLIGA.DK

In [21]:
#1, Combinale all code above in a loop to scrape all pages
add = []
p = []
zipc = []
m_2 = []
rooms_ = []

for i in links:
    response = requests.get(i, headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
    soup = BeautifulSoup(response.content,"xml")
    
    #append address to list
    list_address = soup.find_all("div" ,class_ = 'w-100 mt-1 listing-address font-weight-bold text-nowrap text-truncate')
    address = [col.text.split(",",1)[0] for col in list_address]
    add.append(address)
    #convert list of lists to ONE LIST
    address_final = []
    for sublist in add:
        for item in sublist:
            address_final.append(item)
    
    #append price to list
    list_price = soup.find_all("div" , class_ ="price m-0 d-flex align-items-center")
    price = [col.text.split("k",1)[0] for col in list_price]
    p.append(price)
    #convert list of lists to ONE LIST
    price_final = []
    for sublist in p:
        for item in sublist:
            price_final.append(item)
    
    #append zipcode to list
    list_zipcode = soup.find_all("div" ,class_ = "zip-color text-nowrap text-truncate")
    zipcode = [col.text for col in list_zipcode]
    zipc.append(zipcode)
     #convert list of lists to ONE LIST
    zipcode_final = []
    for sublist in zipc:
        for item in sublist:
            zipcode_final.append(item)
    
    #append m2 to list
    list_m2 = soup.find_all("span" ,class_ = "text-nowrap")
    m2 = [col.text.split() for col in list_m2]
    #the variable m2 is a list of lists. The following code keeps the first element of every sublist to get rid of the string "m2"
    m2 = [element[0] for element in m2]
    del m2[0] #delete first element in m2 in order to get all m2 in next code
    m2 = m2[::6] #we now keep every 6th element, which is the size of every address
    m_2.append(m2)
    #convert list of lists to ONE LIST
    m2_final = []
    for sublist in m_2:
        for item in sublist:
            m2_final.append(item)
    
    #append room to list
    rooms = [col.text.split() for col in list_m2]
    rooms = [elements[1:] for elements in rooms]
    rooms = rooms[::6] #we now keep every 6th element, which is no. of rooms
    rooms_.append(rooms)
    #convert list of lists to ONE LIST
    rooms_fin = []
    for sublist in rooms_:
        for item in sublist:
            rooms_fin.append(item)
    rooms_final = []
    for sublist in rooms_fin:
        for item in sublist:
            rooms_final.append(item)
    #replace "værelse" with "1"
    for i in range(len(rooms_final)):
        if rooms_final[i] == "værelse":
            rooms_final[i] = "1"
    time.sleep(0.5) #Sleep for 0.5 seconds


$\textbf{COMBINE ALL CODE ABOVE INTO ONE DATAFRAME}$

In [33]:
#1. Combine all scraped lists in one dataframe
df_addzip = {"Pris": price_final, "address": address_final, "zipcode": zipcode_final, "m2": m2_final, "rooms": rooms_final}
df_data = pd.DataFrame(df_addzip)

#2. save dataset as v1
df_data.to_csv("v1.csv")
df_data

,Pris,address,zipcode,m2,rooms
0,495.000,Nyvangsvej 18,8900 Randers C,50,2
1,525.000,Dalgas Alle 109,7800 Skive,47,1
2,525.000,Gl. Hobrovej 8 1 tv,8920 Randers NV,66,2
3,595.000,Lyngbyvej 239A,9480 Løkken,46,3
4,749.000,Willemoesgade 7,9000 Aalborg,44,1
...,...,...,...,...,...
7240,4.075.000,Promenaden 32,7000 Fredericia,127,3
7241,4.245.000,Promenaden 31,7000 Fredericia,141,4
7242,4.295.000,Promenaden 34,7000 Fredericia,127,3
7243,295.000,Østkystvejen 51,9300 Sæby,119,4


$\textbf{GET  DISTANCES}, \ dataforsyningen.dk, \ hvorlangterder.dk$

In [34]:
#1. Load pages
Address_zipcode = df_data["address"] + " " + df_data["zipcode"]
add_zip = []
for i in Address_zipcode:
    response_dist = requests.get(f"https://api.dataforsyningen.dk/adresser?q={i}&struktur=mini", headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
    res_j = response_dist.json()
    add_zip.append(res_j)

Before continuing, locate missing values due to typing error in column "address" in df_data

In [35]:
#2. Locate missing values
df_add_zip = pd.DataFrame(add_zip)
index = df_add_zip[df_add_zip[0].isnull()][0]

#3. Store index numbers in order to del the corresponding indexes in df_data
index_list = list(index.index.values)

#4. Delete corresponding rows in df_data
for i in index_list:
    df_add_zip.drop(i, inplace=True)

#5. reset_index keeps failing, so here comes another index-method    
df_add_zip["new_index"] = range(len(df_add_zip))
df_add_zip.set_index("new_index", inplace=True)

Drop missing values in df_data

In [36]:
#6. Delete corresponding rows in df_data
for i in index_list:
    df_data.drop(i, inplace=True)

#reset_index keeps failing, so here comes another index-method    
df_data["new_index"] = range(len(df_data))
df_data.set_index("new_index", inplace=True)

In [38]:
#7. save dataset with dropped missing values as v2
df_data.to_csv("v2.csv")
df_data
#Vi mister 151 observationer efter fjernelse af ovenstående obs

,Pris,address,zipcode,m2,rooms
new_index,,,,,
0,495.000,Nyvangsvej 18,8900 Randers C,50,2
1,525.000,Dalgas Alle 109,7800 Skive,47,1
2,525.000,Gl. Hobrovej 8 1 tv,8920 Randers NV,66,2
3,595.000,Lyngbyvej 239A,9480 Løkken,46,3
4,749.000,Willemoesgade 7,9000 Aalborg,44,1
...,...,...,...,...,...
7089,10.250.000,Franciska Clausens Plads 20,1799 København V,175,3
7090,10.250.000,Franciska Clausens Plads 20,1799 København V,175,3
7091,795.000,Arvidvej 192B,6960 Hvide Sande,73,3


In [39]:
#8. Get longitude, x, and latitudes, y from address i
x = []
y = []

for i in range(len(df_add_zip)):
    xx = df_add_zip[0][i]["x"]
    x.append(xx)

for i in range(len(df_add_zip)):
    yy = df_add_zip[0][i]["y"]
    y.append(yy)

In [40]:
#9. Add coordinates to df_data
df_data["x"] = x
df_data["y"] = y
xy = df_data["y"].apply(str) +","+ df_data["x"].apply(str)

In [41]:
#10 Get distance to nearest x
daycare_distance = []

for i in xy:
    afstand_daycare_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=daycare&decodepolyline')
    afstand_daycare_json.json()
    key = afstand_daycare_json.json()['daycare']
    key = key['routedmeters']
    daycare_distance.append(key)

In [42]:
regionaltog_distance = []

for i in xy:
    afstand_regionaltog_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=train&decodepolyline')
    afstand_regionaltog_json.json()
    key = afstand_regionaltog_json.json()['train']
    key = key['routedmeters']
    regionaltog_distance.append(key)

In [43]:
school_distance = []

for i in xy:
    afstand_school_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=school&decodepolyline')
    afstand_school_json.json()
    key = afstand_school_json.json()['school']
    key = key['routedmeters']
    school_distance.append(key)

In [44]:
supermarket_distance = []

for i in xy:
    afstand_supermarket_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=supermarket&decodepolyline')
    afstand_supermarket_json.json()
    key = afstand_supermarket_json.json()['supermarket']
    key = key['routedmeters']
    supermarket_distance.append(key)

In [45]:
hospital_distance = []

for i in xy:
    hospital_distance_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=hospital&decodepolyline')
    hospital_distance_json.json()
    key = hospital_distance_json.json()['hospital']
    key = key['routedmeters']
    hospital_distance.append(key)

In [46]:
#4 Add coordinates of addresses in dataframe
df_data["daycare_distance"] = daycare_distance
df_data["train_distance"] = regionaltog_distance
df_data["school_distance"] = school_distance
df_data["supermarket_distance"] = supermarket_distance
df_data["hospital_distance"] = hospital_distance

In [48]:
#11. save dataset with distances
df_data.to_csv("v3.csv")
df_data

,Pris,address,zipcode,m2,rooms,x,y,daycare_distance,train_distance,school_distance,supermarket_distance,hospital_distance
new_index,,,,,,,,,,,,
0,495.000,Nyvangsvej 18,8900 Randers C,50,2,10.027365,56.466636,234.05,1197.57,358.13,404.26,2083.77
1,525.000,Dalgas Alle 109,7800 Skive,47,1,9.021491,56.546826,892.49,2126.17,892.49,1217.77,3977.31
2,525.000,Gl. Hobrovej 8 1 tv,8920 Randers NV,66,2,10.019925,56.468845,238.84,1288.30,392.45,259.86,2234.50
3,595.000,Lyngbyvej 239A,9480 Løkken,46,3,9.746882,57.412708,3513.54,17191.77,5792.11,5122.82,19086.03
4,749.000,Willemoesgade 7,9000 Aalborg,44,1,9.899679,57.055060,661.02,914.89,664.37,300.57,1257.92
...,...,...,...,...,...,...,...,...,...,...,...,...
7089,10.250.000,Franciska Clausens Plads 20,1799 København V,175,3,12.534592,55.665601,293.46,1650.66,492.63,279.92,3237.71
7090,10.250.000,Franciska Clausens Plads 20,1799 København V,175,3,12.534592,55.665601,293.46,1650.66,492.63,279.92,3237.71
7091,795.000,Arvidvej 192B,6960 Hvide Sande,73,3,8.174820,55.849587,13414.86,12822.53,12985.61,4784.36,31857.44


$\textbf{CONSTRUCT A VARIABEL FOR REGION}$

In [49]:
#1. Load API
response_reg = requests.get(f"https://api.dataforsyningen.dk/regioner", headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
res_reg = response_reg.json()

In [50]:
#2. Get the coordinate intervals for each region
regions = []

for i in range(0,5):
    rg = res_reg[i]["bbox"]
    regions.append(rg)
regions

[[8.18951662, 56.53454667, 11.22599137, 57.76025478],
 [8.07887623, 55.64437916, 11.66419061, 56.84325702],
 [8.06320315, 54.71828064, 10.99555236, 55.95324986],
 [11.60211644, 54.98354657, 15.31831444, 56.20520188],
 [10.8148054, 54.54440636, 12.64551576, 56.0173058]]

In [51]:
#3. Use the coordinate intervals above to place each observation in a region
def region(df_data):
    
    if (8.18951662 < df_data['x'] < 11.22599137) and (56.53454667 < df_data['y'] < 57.76025478):  
        return 'Nordjylland'
    elif (8.07887623 <  df_data['x'] < 11.66419061) and (55.64437916 < df_data['y'] < 56.84325702): 
        return 'Midtjylland'
    elif (8.06320315 < df_data['x'] < 10.99555236) and (54.71828064 < df_data['y'] < 55.95324986):  
        return 'Syddanmark'
    elif (11.60211644 < df_data['x'] < 15.31831444) and (54.98354657 < df_data['y'] < 56.20520188):  
        return 'Hovedstaden'
    elif (10.8148054 < df_data['x'] < 12.64551576) and (54.54440636 < df_data['y'] < 56.0173058):  
        return 'Sjælland'
df_data['Region'] = df_data.apply(region, axis=1)
#display(df_data)

In [52]:
#4. save dataset including regions
df_data.to_csv("v4.csv")
df_data

,Pris,address,zipcode,m2,rooms,x,y,daycare_distance,train_distance,school_distance,supermarket_distance,hospital_distance,Region
new_index,,,,,,,,,,,,,
0,495.000,Nyvangsvej 18,8900 Randers C,50,2,10.027365,56.466636,234.05,1197.57,358.13,404.26,2083.77,Midtjylland
1,525.000,Dalgas Alle 109,7800 Skive,47,1,9.021491,56.546826,892.49,2126.17,892.49,1217.77,3977.31,Nordjylland
2,525.000,Gl. Hobrovej 8 1 tv,8920 Randers NV,66,2,10.019925,56.468845,238.84,1288.30,392.45,259.86,2234.50,Midtjylland
3,595.000,Lyngbyvej 239A,9480 Løkken,46,3,9.746882,57.412708,3513.54,17191.77,5792.11,5122.82,19086.03,Nordjylland
4,749.000,Willemoesgade 7,9000 Aalborg,44,1,9.899679,57.055060,661.02,914.89,664.37,300.57,1257.92,Nordjylland
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7089,10.250.000,Franciska Clausens Plads 20,1799 København V,175,3,12.534592,55.665601,293.46,1650.66,492.63,279.92,3237.71,Hovedstaden
7090,10.250.000,Franciska Clausens Plads 20,1799 København V,175,3,12.534592,55.665601,293.46,1650.66,492.63,279.92,3237.71,Hovedstaden
7091,795.000,Arvidvej 192B,6960 Hvide Sande,73,3,8.174820,55.849587,13414.86,12822.53,12985.61,4784.36,31857.44,Midtjylland


In [57]:
df_data = df_data.drop_duplicates().reset_index()
df_data.to_csv("v5.csv")